In [ ]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
class CustomDataset(tf.keras.utils.Sequence):
    def __init__(self, df, directory, batch_size=32, random_state=42, shuffle=True, target=True):
        np.random.seed(random_state)
        
        self.directory = directory
        self.df = df
        self.shuffle = shuffle
        self.target = target
        self.batch_size = batch_size
        self.ext = '.npy'
        
        self.on_epoch_end()
    
    def __len__(self):
        return np.ceil(self.df.shape[0] / self.batch_size).astype(int)
    
    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        batch = self.df[start_idx: start_idx + self.batch_size]
        
        signals = []

        for fname in batch.id:
            path = os.path.join(self.directory, fname[0], fname + self.ext)
            data = np.load(path)
            signals.append(data)
        
        signals = np.transpose(np.stack(signals), (0, 1, 3, 2)).astype('float32')
        
        if self.target:
            return signals, batch.target.values
        else:
            return signals
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
def build_model():
    inputs = layers.Input(shape=(6, 256, 273))

    gru1 = layers.Bidirectional(layers.GRU(128, return_sequences=True))
    gru2 = layers.Bidirectional(layers.GRU(128, return_sequences=True))
    pool = layers.GlobalAveragePooling1D()

    x = layers.TimeDistributed(gru1, name="bi_gru_1")(inputs)
    x = layers.Dropout(0.5)(x)
    x = layers.TimeDistributed(gru2, name="bi_gru_2")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.TimeDistributed(pool, name="pool")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation="relu")(x)#128
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(1, activation="sigmoid", name="sigmoid")(x)#sigmoid

    model = tf.keras.Model(inputs=inputs, outputs=x)
    
    return model

In [ ]:
train = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
sub = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
train.head()

In [ ]:
sample_df = train.sample(frac=1).reset_index(drop=True)

split = int(sample_df.shape[0] * 0.8)
train_df = sample_df[:split]
valid_df = sample_df[split:]

In [ ]:
model = build_model()
model.compile("adam", loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC()])
model.summary()

In [ ]:
train_dset = CustomDataset(
    train_df, "../input/seti-breakthrough-listen/train", batch_size=64)

valid_dset = CustomDataset(
    valid_df, "../input/seti-breakthrough-listen/train", batch_size=64, shuffle=False)

test_dset = CustomDataset(
    sub, "../input/seti-breakthrough-listen/test", batch_size=64, target=False, shuffle=False)

In [ ]:
# from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
ckpt = tf.keras.callbacks.ModelCheckpoint(
    "model_weights.h5", save_best_only=True, save_weights_only=True,
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001, verbose=1)

train_history = model.fit(
    train_dset, 
    use_multiprocessing=True, 
    workers=-1,#4 
    epochs=20,
    validation_data=valid_dset,
    callbacks=[ckpt,reduce_lr],
)

In [ ]:
import plotly.express as px
px.line(train_history.history, y=['auc', 'val_auc'], title="Training history")

## Submission

In [ ]:
model.load_weights('model_weights.h5')

In [ ]:
y_pred = model.predict(
    test_dset, use_multiprocessing=True, workers=-1, verbose=1
)

In [ ]:
sub['target'] = y_pred

In [ ]:
sub.to_csv('submission.csv', index=False)